In [11]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms

from sklearn.ensemble import RandomForestClassifier

import numpy as np
import pandas as pd

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [67]:
batch_size = 64
num_epochs = 5
workers = 2
lr = 1e-3

input_size = 28*28
latent_dim = 2
num_classes = 10

In [50]:
transform = transforms.Compose([
                                transforms.ToTensor(),
])

In [51]:
train_dataset = torchvision.datasets.MNIST(root = '../../data',
                                           train = True,
                                           download = True,
                                           transform = transform,
                                           )

test_dataset = torchvision.datasets.MNIST(root = '../../data',
                                          train = False,
                                          transform = transform,
                                          )

In [59]:
train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size = batch_size,
                                               shuffle = True,
                                               num_workers = workers,
                                               )
test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                              batch_size = batch_size,
                                              shuffle = False,
                                              num_workers = workers,
                                              )

In [25]:
sample, labels = iter(train_dataloader).next()
sample.shape, labels.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

In [68]:
class AE(nn.Module):
    def __init__(self):
        super(AE, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(in_features=input_size, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=32),
            nn.ReLU(),
            nn.Linear(in_features=32, out_features=latent_dim),
        )
        self.decoder = nn.Sequential(
            nn.Linear(in_features=latent_dim, out_features=32),
            nn.ReLU(),
            nn.Linear(in_features=32, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=input_size),
            nn.ReLU(),
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [69]:
AE_model = AE().to(device)

In [70]:
# Loss and Optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(AE_model.parameters(), lr=lr)

In [71]:
print(AE_model)

AE(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=2, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=2, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=784, bias=True)
    (5): ReLU()
  )
)


In [64]:
from tqdm import tqdm

In [76]:
# Train AE
total_step = len(train_dataloader)
train_step = 1

log_interval = 500
for epoch in range(1, num_epochs+1):
    encoded_list = []
    label_list = []
    for batch_idx, (images, labels) in enumerate(
        tqdm(train_dataloader, position=0, leave=True, desc='train')
    ):
        images = images.view(-1, input_size).to(device)
        targets = images.view(-1, input_size).to(device)

        # Forward
        encoded, decoded = AE_model(images)
        loss = criterion(decoded, targets)

        ## Appending for ML
        encoded_list.append(encoded)
        label_list.append(labels)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Logging
        if train_step%log_interval == 0:
            print(f'Epoch [{epoch}/{num_epochs}] Step [{train_step}/{total_step*num_epochs}] Loss : {loss.item():.4f}')
        train_step += 1

train:  55%|█████▍    | 514/938 [00:04<00:04, 103.49it/s]

Epoch [1/5] Step [500/4690] Loss : 0.0399


train:   8%|▊         | 72/938 [00:01<00:10, 80.05it/s]

Epoch [2/5] Step [1000/4690] Loss : 0.0409


train:  61%|██████▏   | 575/938 [00:08<00:04, 82.03it/s]

...

train:  28%|██▊       | 262/938 [00:02<00:06, 97.60it/s]

Epoch [5/5] Step [4000/4690] Loss : 0.0430


train:  81%|████████▏ | 763/938 [00:07<00:01, 113.20it/s]

Epoch [5/5] Step [4500/4690] Loss : 0.0357


train: 100%|██████████| 938/938 [00:08<00:00, 105.91it/s]


In [189]:
# Create Train csv

df = pd.DataFrame(columns=['x1', 'x2', 'target'])

for idx, (data, target) in enumerate(
    tqdm(zip(encoded_list, label_list), desc='to_csv')
    ):
    data = data.cpu().detach().numpy()
    target = target.cpu().detach().numpy()

    temp = pd.DataFrame(data, columns=['x1', 'x2'])
    temp['target'] = target
    df = pd.concat((df, temp), axis=0)

df = df.astype({'x1':np.float32, 'x2':np.float32, 'target':np.int8})
print()
print(df['target'].dtype)

to_csv: 938it [00:01, 600.10it/s]


int8


In [126]:
AE_model.eval()
test_encoded_list = []
test_label_list = []
with torch.no_grad():
    for test_images, test_labels in test_dataloader:
        test_images = test_images.view(-1, input_size).to(device)
        test_labels = test_labels.to(device)
        test_encoded, test_decoded = AE_model(test_images)

        test_encoded_list.append(test_encoded)
        test_label_list.append(test_labels)

In [190]:
# Create Test csv

test_df = pd.DataFrame(columns=['x1', 'x2', 'target'])

for idx, (data, target) in enumerate(
    tqdm(zip(test_encoded_list, test_label_list), desc='to_csv')
    ):
    data = data.cpu().detach().numpy()
    target = target.cpu().detach().numpy()

    temp = pd.DataFrame(data, columns=['x1', 'x2'])
    temp['target'] = target
    test_df = pd.concat((test_df, temp), axis=0)

test_df = test_df.astype({'x1':np.float32, 'x2':np.float32, 'target':np.int8})
print()
print(test_df['target'].dtype)

to_csv: 157it [00:00, 817.31it/s]


int8


# Check csv

In [191]:
df

,x1,x2,target
0,-1.637074,1.197929,1
1,1.935903,-0.740821,8
2,2.418931,-0.283956,4
3,2.417864,-1.452230,3
4,0.367098,1.280773,7
...,...,...,...
27,-0.352888,-5.033825,0
28,4.813618,3.457771,7
29,1.403693,-1.328049,3
30,0.181385,0.925137,4


In [192]:
test_df

,x1,x2,target
0,2.365258,2.080255,7
1,-1.057445,0.003739,2
2,-0.897244,1.046088,1
3,0.877949,-2.320749,0
4,3.905225,0.519765,4
...,...,...,...
11,-2.031609,-0.378456,2
12,1.545131,-1.542950,3
13,0.686410,0.666762,4
14,-0.211429,-0.041775,5


# RF

In [204]:
X_train, y_train = df.loc[:, ['x1', 'x2']], df.loc[:, ['target']]
X_test, y_test = test_df.loc[:, ['x1', 'x2']], test_df.loc[:, ['target']]

In [205]:
RF = RandomForestClassifier(random_state=42, n_jobs=-1)
RF.fit(X_train, y_train.to_numpy().reshape(-1,))

RandomForestClassifier(n_jobs=-1, random_state=42)

In [206]:
pred = RF.predict(X_test)
acc = np.sum(y_test.to_numpy().reshape(-1,) == pred) / pred.shape[0]

In [207]:
acc

0.7859